In [1]:
import tensorflow

tensorflow.random.set_random_seed(5)
from numpy.random import seed

seed(5)
import time
import numpy as np

np.random.seed(5)

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import r2_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from tensorflow.python.keras.utils.vis_utils import plot_model

from math import sqrt
import pandas as pd

from tensorflow.python.keras.layers import merge
from tensorflow.python.keras.layers.core import *
from tensorflow.python.keras.layers.recurrent import LSTM
from tensorflow.python.keras.models import *
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Input, Dropout, Flatten, MaxPooling1D, TimeDistributed, GRU, SimpleRNN, \
    Bidirectional, RNN
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.python.framework.ops import disable_eager_execution
tf.test.is_gpu_available()

True

In [2]:
disable_eager_execution()
china_v = pd.read_csv('D:\project\pythonProject2\Japan.csv')
china_data = china_v['value']

In [3]:
len(china_v[:1610])

1610

In [4]:
china_v1 = china_data[:1610]
data = np.array(china_v1)
data = data.reshape(-1, 1)

In [5]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

In [6]:
china_v1

0       1.295800
1       1.272510
2       1.327940
3       1.470240
4       1.483790
          ...   
1605    1.089140
1606    1.101800
1607    1.108370
1608    0.943891
1609    0.978252
Name: value, Length: 1610, dtype: float64

进行平稳性检测

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(china_v1).show()

通过自相关图来进行检验，对于平稳时间序列，其自相关图一般随着阶数的递增，自相关系统会迅速衰减至0附近，而非平稳时间序列则可能存在先减后增或者周期性波动等变动。如下图所示，该时间序列随着阶数的递增，自相关系数先减后增，因此，可以判断该时间序列不是平稳时间序列。

In [ ]:
#定义ADF输出格式化函数
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    print ('ADF检验结果:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Number of Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

#对数据集使用ADF检验
adf_test(china_v1)


白噪声检验

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
acorr_ljungbox(china_v1, lags=None, boxpierce=False)

利用差分法对时间序列进行平稳化

In [ ]:
def first_diff(data):
    """
    计算给定数据列表的相邻元素之间的差值。

    参数:
    data: 一个包含数字元素的列表或数组，可以是浮点数或整数。

    返回值:
    一个numpy数组，包含输入数据中相邻元素之间的差值，数据类型为float64。
    """
    # 初始化用于存储差值的列表
    data_diff = []
    # 计算相邻元素之间的差值，并添加到data_diff列表中
    for i in range(len(data) - 1):
        data_diff.append(data[i + 1] - data[i])
    # 将data_diff列表转换为numpy数组
    data_diff = np.array(data_diff)
    # 将数组的数据类型转换为float64并返回
    return data_diff.astype(np.float64)

差分后的自相关图

In [ ]:
tec_diff= first_diff(np.array(data))
# adf_test(tec_diff)
# show=[]
# for i in range(len(tec_diff)-1):
#     show.append(tec_diff[i][0])
# show=np.array(show)
# ser=pd.Series(show)
# ser.plot()
# plt.show()

In [ ]:
len(tec_diff)

划分训练集和测试集，用前70%的数据作为训练集，后30%的数据作为测试集

In [ ]:
split_size = int(len(tec_diff) * 0.7)
train, test = tec_diff[:split_size], tec_diff[split_size:]

In [ ]:
scaler = MinMaxScaler()# 归一化到[0,1] 转换函数：x = (x-min) / (max-min)；适用性：适用于分布范围较稳定的数据，当新数据的加入导致max/min变化，则需重新定义；
scaled_train = scaler.fit_transform(train)
scaled_test = scaler.transform(test)

In [ ]:
def split_sequence(sequence, look_back, forecast_horizon):
    X, y = list(), list()
    for i in range(len(sequence)):
        lag_end = i + look_back
        forecast_end = lag_end + forecast_horizon
        if forecast_end > len(sequence):
            break
        seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
LOOK_BACK = 12
FORECAST_RANGE = 1
x_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)

In [ ]:
len(x_train),len(y_train)

In [ ]:
x_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
xa, ya = split_sequence(np.array(data[int(len(tec_diff) * 0.7) + 1:]), look_back=LOOK_BACK,
                        forecast_horizon=FORECAST_RANGE)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=20)  # 早停回调
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
INPUT_DIMS = 1
TIME_STEPS = 12
start = time.perf_counter()
tf.random.set_random_seed(5)

搭建模型

In [ ]:
def attention_3d_block(inputs):
    a = Permute((2, 1))(inputs)
    a = Dense(LOOK_BACK, activation='softmax')(a)
    a_probs = Permute((2, 1))(a)
    # 相当于获得每一个step中，每个特征的权重
    output_attention_mul = merge.multiply([inputs, a_probs])  # 新版本 keras
    return output_attention_mul
def get_attention_model(time_steps, input_dim, lstm_units=64):
    inputs1 = Input(shape=(time_steps, input_dim))
    lstm_out = GRU(lstm_units, return_sequences=True)(inputs1)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    # attention_mul = Flatten()(lstm_out)
    output = Dense(1)(attention_mul)
    model = Model(inputs=[inputs1], outputs=output)
    return model
model = get_attention_model(LOOK_BACK, INPUT_DIMS)

优化器

In [ ]:
from sklearn.model_selection import GridSearchCV


def get_data_recurrent(x, y, attention_column=7):
    y = y.reshape(y.shape[0], 1)
    x[:, attention_column, :] = np.tile(y[:], (1, 1))
    return x, y
X, Y = get_data_recurrent(x_train, y_train)
X_test, Y_test = get_data_recurrent(x_test, y_test)
epochs=20
model.compile(optimizer='SGD', loss='mae')
history = model.fit(X, Y, epochs=epochs, callbacks=[early_stop], batch_size=64, verbose=2, shuffle=False,
                    validation_split=0.1)
# model.summary()
yhat = model.predict(x_test, verbose=0)
def inverse_transform(y_test, yhat):
    y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])
    yhat_reshaped = yhat.reshape(-1, yhat.shape[-1])
    yhat_inverse = scaler.inverse_transform(yhat_reshaped)
    y_test_inverse = scaler.inverse_transform(y_test_reshaped)
    return yhat_inverse, y_test_inverse
def anti_first_diff(m, n):
    data = []
    for i in range(0, len(m)):
        a = n[i] + m[i]
        data.append(a)
    return np.array(data).astype("float32")
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)  # 反转归一化
y_pre = []  #
print(y_test_inverse)
for i in range(yhat_inverse.shape[0]):
    for j in range(yhat_inverse.shape[1]):
        y_pre.append(yhat_inverse[i][j])
y = np.array(y_test_inverse)
y_pre = np.array(y_pre)  # 将y_pre转化为numpy数组
y = anti_first_diff(ya, y)
y_pre = anti_first_diff(ya, y_pre)

# y_pre = np.array(y_pre)
y_r, y_p = [], []
for i in range(0, y.shape[0]):
    for j in range(0, y.shape[1]):
        y_r.append(y[i][j])
        y_p.append(y_pre[i][j])# y_r

y_r1 = np.array(y_r)
y_p1 = np.array(y_p)
y_r2 = data[1139:]
x = np.arange(0, 1610, 1)
MAE = mean_absolute_error(y_r2, y_p1)
MSE = mean_squared_error(y_r2, y_p1)
RMSE = sqrt(mean_squared_error(y_r2, y_p1))
R2_score = r2_score(y_r2, y_p1)
output_value = []
output_value.append(MAE)
output_value.append(MSE)
output_value.append(RMSE)
output_value.append(R2_score)
# output_value.append(int(using_time))
output_value = np.array(output_value)
output_data = pd.DataFrame(output_value)

output_data.index = ['MAE', 'MSE', 'RMSE', 'R2 Score']
print(output_data)

In [ ]:
qweqwe

In [ ]:
SGD = get_attention_model(LOOK_BACK, INPUT_DIMS)
Adam = get_attention_model(LOOK_BACK, INPUT_DIMS)
# Adagrad = get_attention_model(LOOK_BACK, INPUT_DIMS)
Nadam = get_attention_model(LOOK_BACK, INPUT_DIMS)

In [ ]:
sgd16=get_attention_model(LOOK_BACK, INPUT_DIMS)
sgd32=get_attention_model(LOOK_BACK, INPUT_DIMS)
sgd64=get_attention_model(LOOK_BACK, INPUT_DIMS)
sgd128=get_attention_model(LOOK_BACK, INPUT_DIMS)
sgd256=get_attention_model(LOOK_BACK, INPUT_DIMS)

In [ ]:
sgd16.compile(optimizer='SGD', loss='mae')
sgd_16=sgd16.fit(X, Y, epochs=epochs, batch_size=16, verbose=2, shuffle=False,
                    validation_split=0.1)
sgd32.compile(optimizer='SGD', loss='mae')
sgd_32=sgd32.fit(X, Y, epochs=epochs, batch_size=32, verbose=2, shuffle=False,
                    validation_split=0.1)
sgd64.compile(optimizer='SGD', loss='mae')
sgd_64=sgd64.fit(X, Y, epochs=epochs, batch_size=64, verbose=2, shuffle=False,
                    validation_split=0.1)
sgd128.compile(optimizer='SGD', loss='mae')
sgd_128=sgd128.fit(X, Y, epochs=epochs, batch_size=128, verbose=2, shuffle=False,
                    validation_split=0.1)
sgd256.compile(optimizer='SGD', loss='mae')
sgd_256=sgd256.fit(X, Y, epochs=epochs, batch_size=256, verbose=2, shuffle=False,
                    validation_split=0.1)

In [ ]:
Adam.compile(optimizer='Adam', loss='mae')
adam=Adam.fit(X, Y, epochs=epochs, batch_size=64, verbose=2, shuffle=False,
                    validation_split=0.1)

In [ ]:
Nadam.compile(optimizer='Nadam', loss='mae')
nadam=Nadam.fit(X, Y, epochs=epochs, batch_size=64, verbose=2, shuffle=False,
                    validation_split=0.1)
# plt.plot(sgd.history['loss'],label='SGD')
# plt.plot(nadam.history['loss'],label='Nadam')
# plt.plot(adam.history['loss'],label='Adam')
# plt.plot(history.history['loss'],label='Adagrad')
#
# plt.legend()

In [ ]:
# plot_model(,to_file='model.png',show_shapes=True)

测试集预测

In [ ]:
yhat = model.predict(x_test, verbose=0)

In [ ]:
def inverse_transform(y_test, yhat):
    y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])
    yhat_reshaped = yhat.reshape(-1, yhat.shape[-1])
    yhat_inverse = scaler.inverse_transform(yhat_reshaped)
    y_test_inverse = scaler.inverse_transform(y_test_reshaped)
    return yhat_inverse, y_test_inverse
def anti_first_diff(m, n):
    data = []
    for i in range(0, len(m)):
        a = n[i] + m[i]
        data.append(a)
    return np.array(data).astype("float32")
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)  # 反转归一化
y_pre = []  #
print(y_test_inverse)
for i in range(yhat_inverse.shape[0]):
    for j in range(yhat_inverse.shape[1]):
        y_pre.append(yhat_inverse[i][j])
y = np.array(y_test_inverse)
y_pre = np.array(y_pre)  # 将y_pre转化为numpy数组
y = anti_first_diff(ya, y)
y_pre = anti_first_diff(ya, y_pre)

# y_pre = np.array(y_pre)
y_r, y_p = [], []
for i in range(0, y.shape[0]):
    for j in range(0, y.shape[1]):
        y_r.append(y[i][j])
        y_p.append(y_pre[i][j])# y_r


In [ ]:
y_r1 = np.array(y_r)
y_p1 = np.array(y_p)
y_r2 = data[1139:]
x = np.arange(0, 1610, 1)
MAE = mean_absolute_error(y_r2, y_p1)
MSE = mean_squared_error(y_r2, y_p1)
RMSE = sqrt(mean_squared_error(y_r2, y_p1))
R2_score = r2_score(y_r2, y_p1)
output_value = []
output_value.append(MAE)
output_value.append(MSE)
output_value.append(RMSE)
output_value.append(R2_score)
# output_value.append(int(using_time))
output_value = np.array(output_value)
output_data = pd.DataFrame(output_value)

output_data.index = ['MAE', 'MSE', 'RMSE', 'R2 Score']
print(output_data)

In [ ]:
def modelpre(yhat):
    yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)  # 反转归一化
    y_pre = []  #
    for i in range(yhat_inverse.shape[0]):
        for j in range(yhat_inverse.shape[1]):
            y_pre.append(yhat_inverse[i][j])
    y = np.array(y_test_inverse)
    y_pre = np.array(y_pre)  # 将y_pre转化为numpy数组
    y = anti_first_diff(ya, y)
    y_pre = anti_first_diff(ya, y_pre)

    # y_pre = np.array(y_pre)
    y_r, y_p = [], []
    for i in range(0, y.shape[0]):
        for j in range(0, y.shape[1]):
            y_r.append(y[i][j])
            y_p.append(y_pre[i][j])# y_r
    y_r1 = np.array(y_r)
    y_p1 = np.array(y_p)
    y_r2 = data[1139:]
    x = np.arange(0, 1610, 1)
    MAE = mean_absolute_error(y_r2, y_p1)
    MSE = mean_squared_error(y_r2, y_p1)
    RMSE = sqrt(mean_squared_error(y_r2, y_p1))
    R2_score = r2_score(y_r2, y_p1)
    output_value = []
    output_value.append(MAE)
    output_value.append(MSE)
    output_value.append(RMSE)
    output_value.append(R2_score)
    # output_value.append(int(using_time))
    output_value = np.array(output_value)
    output_data = pd.DataFrame(output_value)

    output_data.index = ['MAE', 'MSE', 'RMSE', 'R2 Score']
    print(output_data)
    return y_r2, y_p1,MSE, RMSE,MAE,R2_score

Adagrad

In [ ]:
adagrad_pre = model.predict(x_test, verbose=0)
adagrad_pre_yhat,adagrad_pre_y_r2,adagrad_pre_MSE, adagrad_pre_RMSE,adagrad_pre_MAE,adagrad_pre_R2=modelpre(adagrad_pre)

SGD

In [ ]:
SGD_pre = SGD.predict(x_test, verbose=0)
SGD_pre_yhat,SGD_pre_y_r2,SGD_pre_MSE, SGD_pre_RMSE,SGD_pre_MAE,SGD_pre_R2=modelpre(SGD_pre)

NAdam

In [ ]:
NAdam_pre = Nadam.predict(x_test, verbose=0)
NAdam_pre_yhat,NAdam_pre_y_r2,NAdam_pre_MSE, NAdam_pre_RMSE,NAdam_pre_MAE,NAdam_pre_R2=modelpre(NAdam_pre)

adam

In [ ]:
Adam_pre = Adam.predict(x_test, verbose=0)
Adam_pre_yhat,Adam_pre_y_r2,Adam_pre_MSE, Adam_pre_RMSE,Adam_pre_MAE,Adam_pre_R2=modelpre(Adam_pre)

In [ ]:
MAE=[]
MAE.append(adagrad_pre_MAE)
MAE.append(Adam_pre_MAE)
MAE.append(NAdam_pre_MAE)
MAE.append(SGD_pre_MAE)
plt.title('MAE')
plt.bar(range(4),MAE,align = "center",color = "steelblue",alpha = 1.0)
plt.xticks(range(4),['Adagrad','Adam','Nadam','SGD'])
plt.show()

In [ ]:
MSE=[]
MSE.append(adagrad_pre_MSE)
MSE.append(Adam_pre_MSE)
MSE.append(NAdam_pre_MSE)
MSE.append(SGD_pre_MSE)
plt.title('MSE')
plt.bar(range(4),MSE,align = "center",color = "steelblue",alpha = 1.0)
plt.xticks(range(4),['Adagrad','Adam','Nadam','SGD'])
plt.show()

In [ ]:
RMSE=[]
RMSE.append(adagrad_pre_RMSE)
RMSE.append(Adam_pre_RMSE)
RMSE.append(NAdam_pre_RMSE)
RMSE.append(SGD_pre_RMSE)
plt.title('RMSE')
plt.bar(range(4),RMSE,align = "center",color = "steelblue",alpha = 1.0)
plt.xticks(range(4),['Adagrad','Adam','Nadam','SGD'])
plt.show()

In [ ]:
type(RMSE)

In [ ]:
R2=[]
R2.append(adagrad_pre_R2)
R2.append(Adam_pre_R2)
R2.append(NAdam_pre_R2)
R2.append(SGD_pre_R2)
plt.title('R2_score')
plt.bar(range(4),R2,align = "center",color = "steelblue",alpha = 1.0)
plt.xticks(range(4),['Adagrad','Adam','Nadam','SGD'])
plt.show()

In [ ]:
sgd_16_pre=sgd16.predict(x_test, verbose=0)
sgd_16_pre_yhat,sgd_16_pre_y_r2,sgd_16_pre_MSE, sgd_16_pre_RMSE,sgd_16_pre_MAE,sgd_16_pre_R2=modelpre(sgd_16_pre)
sgd_32_pre=sgd32.predict(x_test, verbose=0)
sgd_32_pre_yhat,sgd_32_pre_y_r2,sgd_32_pre_MSE, sgd_32_pre_RMSE,sgd_32_pre_MAE,sgd_32_pre_R2=modelpre(sgd_32_pre)
sgd_64_pre=sgd64.predict(x_test, verbose=0)
sgd_64_pre_yhat,sgd_64_pre_y_r2,sgd_64_pre_MSE, sgd_64_pre_RMSE,sgd_64_pre_MAE,sgd_64_pre_R2=modelpre(sgd_64_pre)
sgd_128_pre=sgd128.predict(x_test, verbose=0)
sgd_128_pre_yhat,sgd_128_pre_y_r2,sgd_128_pre_MSE, sgd_128_pre_RMSE,sgd_128_pre_MAE,sgd_128_pre_R2=modelpre(sgd_128_pre)
sgd_256_pre=sgd256.predict(x_test, verbose=0)
sgd_256_pre_yhat,sgd_256_pre_y_r2,sgd_256_pre_MSE, sgd_256_pre_RMSE,sgd_256_pre_MAE,sgd_256_pre_R2=modelpre(sgd_256_pre)

In [ ]:
RMSE_betch_size=[]
RMSE_betch_size.append(sgd_16_pre_RMSE)
RMSE_betch_size.append(sgd_32_pre_RMSE)
RMSE_betch_size.append(sgd_64_pre_RMSE)
RMSE_betch_size.append(sgd_128_pre_RMSE)
RMSE_betch_size.append(sgd_256_pre_RMSE)
plt.title('RMSE')
plt.bar(range(5),RMSE_betch_size,align = "center",color = "steelblue",alpha = 1.0)
plt.xticks(range(5),['16','32','64','128','256'])
plt.show()